<a href="https://www.kaggle.com/code/kentvejrupmadsen/tensorflow-image-classification-of-letters?scriptVersionId=133539842" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install ipykernel
!python -m ipykernel install --user --name=notebook_environment

%pip install matplotlib
%pip install keras
%pip install tensorflow
%pip install kaggle
%pip install numpy
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Installed kernelspec notebook_environment in /root/.local/share/jupyter/kernels/notebook_environment
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from random \
    import SystemRandom

In [3]:
import tensorflow

from tensorflow.data \
    import AUTOTUNE

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
from keras.losses \
    import SparseCategoricalCrossentropy

from keras.utils \
    import image_dataset_from_directory

from keras \
    import \
    Model, \
    Sequential

from keras \
    import layers

In [5]:
import wandb

from wandb.integration.keras \
    import \
    WandbCallback, \
    WandbEvalCallback, \
    WandbMetricsLogger, \
    WandbModelCheckpoint

In [6]:
strategy = tensorflow.distribute.MirroredStrategy(
    devices=["/gpu:0", "/gpu:1"]
)

In [7]:
use_cache: bool = False
log_runs: bool = True

In [8]:
if log_runs:
    wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
r_generator = SystemRandom()

In [10]:
cache_buffer_size: int = 1

In [11]:
configuration = {
    'batch_size': 14,
    'epochs': 10,
    
    'view': {
        'height': 512,
        'width': 512
    },
    
    'dataset':
    {
        'split_validation_at': 0.25,
        
        'training':
        {
            'labels': [],
            'size': 0
        },
        
        'validation':
        {
            'labels': [],
            'size': 0
        }
    },
    
    'seed': r_generator.randint(1, 750000)
}

In [12]:
vision_height: int = configuration['view']['height']
vision_width: int = configuration['view']['width']

vision_size = (vision_height, vision_width) 
vision_size_w_channels = (vision_height, vision_width, 3)

In [13]:
split_dataset_at: float = configuration['dataset']['split_validation_at']
dataset_seed: int = configuration['seed']
dataset_batch_size: int = configuration['batch_size']

In [14]:
path_to_dataset: str = '../input/letter-images-dataset/dataset'

In [15]:
training_dataset = image_dataset_from_directory(
    path_to_dataset,
    validation_split = split_dataset_at,
    subset='training',
    seed = dataset_seed,
    image_size = vision_size,
    batch_size = dataset_batch_size,
    shuffle=True,
    crop_to_aspect_ratio=True
)

validation_dataset = image_dataset_from_directory(
    path_to_dataset,
    validation_split = split_dataset_at,
    subset='validation',
    seed = dataset_seed,
    image_size = vision_size,
    batch_size = dataset_batch_size,
    shuffle=True,
    crop_to_aspect_ratio=True
)


Found 11339 files belonging to 67 classes.
Using 8505 files for training.
Found 11339 files belonging to 67 classes.
Using 2834 files for validation.


In [16]:
configuration['dataset']['training']['size'] = len(training_dataset.class_names)
configuration['dataset']['training']['labels'] = training_dataset.class_names

configuration['dataset']['validation']['size'] = len(validation_dataset.class_names)
configuration['dataset']['validation']['labels'] = validation_dataset.class_names

In [17]:
if log_runs:
    wandb.init(
        project='letter-identification',
        entity='designermadsen', 
        config=configuration,
        sync_tensorboard=True,
        save_code=True,
        monitor_gym=True
    )

wandb: Currently logged in as: designermadsen. Use `wandb login --relogin` to force relogin


In [18]:
category_size = configuration['dataset']['training']['size'] 

In [19]:
def make_model():
    global category_size, vision_size_w_channels
    
    division = (2, 2)
    
    return Sequential(
            [
                layers.Rescaling(
                    1./255, 
                    input_shape=vision_size_w_channels, 
                    trainable=False
                ),
                
                layers.Conv2D(
                    256, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),
                
                layers.Conv2D(
                    128, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3,
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64,
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    64, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    32, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),

                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.Conv2D(
                    16, 
                    3, 
                    padding='same', 
                    activation='relu'
                ),
                
                layers.MaxPooling2D(
                    pool_size=division
                ),
                
                layers.Flatten(),
                layers.Dense(
                    (category_size * 8), 
                    activation='relu',
                ),
                
                layers.Dense(
                    category_size
                )
            ]
        )

In [20]:
if not (strategy is None):
    with strategy.scope():    
        model = make_model()
else:
    model = make_model()

In [21]:
model.compile(
    optimizer='adam', 
    loss=SparseCategoricalCrossentropy(
        from_logits=True
    ), 
    metrics=['accuracy'], 
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 512, 512, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 512, 512, 256)     7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 256, 256)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 128)     295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 128, 128, 128)    0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 64)      7

In [22]:
pointer_to_training_dataset = None
pointer_to_validation_dataset = None

In [23]:
if use_cache:
    pointer_to_training_dataset = training_dataset.cache().prefetch(
        buffer_size=cache_buffer_size
    )
    
    pointer_to_validation_dataset = validation_dataset.cache().prefetch(
        buffer_size=cache_buffer_size
    )
else:
    pointer_to_training_dataset = training_dataset
    pointer_to_validation_dataset = validation_dataset

In [24]:
wandb_callback = WandbCallback(
    log_weights=True, 
    log_gradients=True, 
    training_data=pointer_to_training_dataset, 
    validation_data=pointer_to_validation_dataset,
    log_evaluation=True, 
    
    monitor='val_accuracy',
    mode='max'
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [25]:
epochs = configuration['epochs']
fit_callbacks = []

if not (wandb_callback is None):
    fit_callbacks.append(wandb_callback)

In [26]:
history = model.fit(
    pointer_to_training_dataset,
    validation_data = pointer_to_validation_dataset,
    epochs=epochs,
    callbacks=fit_callbacks
)

wandb: WARNING Error initializing ValidationDataLogger in WandbCallback. Skipping logging validation data. Error: '_BatchDataset' object is not subscriptable


Epoch 1/10
608/608 [==============================] - ETA: 0s - loss: 3.8781 - accuracy: 0.0606

wandb: Adding directory to artifact (/kaggle/working/wandb/run-20230612_225432-p8sa6dh9/files/model-best)... Done. 0.0s


608/608 [==============================] - 600s 934ms/step - loss: 3.8781 - accuracy: 0.0606 - val_loss: 3.8772 - val_accuracy: 0.0547
Epoch 2/10
608/608 [==============================] - 583s 958ms/step - loss: 3.8512 - accuracy: 0.0622 - val_loss: 3.8727 - val_accuracy: 0.0547
Epoch 3/10
608/608 [==============================] - 554s 911ms/step - loss: 3.8486 - accuracy: 0.0641 - val_loss: 3.8710 - val_accuracy: 0.0547
Epoch 4/10
608/608 [==============================] - 555s 912ms/step - loss: 3.8469 - accuracy: 0.0641 - val_loss: 3.8697 - val_accuracy: 0.0547
Epoch 5/10
608/608 [==============================] - 553s 910ms/step - loss: 3.8455 - accuracy: 0.0641 - val_loss: 3.8699 - val_accuracy: 0.0547
Epoch 6/10
608/608 [==============================] - 560s 921ms/step - loss: 3.8452 - accuracy: 0.0641 - val_loss: 3.8690 - val_accuracy: 0.0547
Epoch 7/10
608/608 [==============================] - 557s 915ms/step - loss: 3.8443 - accuracy: 0.0641 - val_loss: 3.8686 - val_accura

In [ ]:
!ls /kaggle

In [27]:
result = {
    'values': {
        'accuracy': history.history['val_accuracy'],
        'loss': history.history['val_loss']
    },
    'result': {
        'accuracy': history.history['accuracy'],
        'loss': history.history['loss']
    }
}

wandb.log(result)

NameError: name 'a' is not defined

In [ ]:
wandb.finish()